# Fetch data from the ARGO drifter program


[ARGO drifters](https://argo.ucsd.edu/faq/#look) are carefully designed scientific instruments.
Physically they are cylinders about 20 cm in diameter and 1.3 m in length. They use a buoyancy control 
bladder to regulate their depth and they measure temperature, salinity, pressure and additional
biogeochemical observables within the ocean. ARGO drifters typically float in the deep ocean
at a depth of 1000 meters. After residing at that depth for ten days they descend to 2000 m and 
then ascend through the water column to the surface where they report back via satellite link. 
Then they return to 1000 meters and the process repeats for the life of the drifter, typically 
about four years. ARGO drifters are subject to ocean currents: They go where the ocean takes them. 


Our goal here is to review the process of obtaining ARGO data as 'scientist data customers'. 


## Obtain ARGO data

ARGO drifters are distributed throughout the global ocean. Each periodically telemeters recent data 
(say 10 days worth) back to a data system. This section briefly reviews searching for and obtaining
data from this system. As with the OOI system it is very much an "order your data / wait for confirmation / 
download your order using ftp" paradigm.

### Search using correct date format dd-mm-yyyy

* Coincident with the work in this notebook: 1-JUN-2019 to 1-SEP-2019
  * ftp://ftp.ifremer.fr/ifremer/coriolis/tmp/co0501/DataSelection_20200129_183035_9538028.tgz
* Full RCA time extent from Aug 1 2014
  * 1-AUG-2014 to 1-JAN-2016
  * ftp://ftp.ifremer.fr/ifremer/coriolis/tmp/co0501/DataSelection_20200129_190230_9538576.tgz
  * 1-JAN-2016 to 1-JAN-2018
  * ftp://ftp.ifremer.fr/ifremer/coriolis/tmp/co0501/DataSelection_20200129_190438_9538617.tgz
  * 1-JAN-2018 to 28-JAN-2020
  * ftp://ftp.ifremer.fr/ifremer/coriolis/tmp/co0501/DataSelection_20200129_190718_9538702.tgz
  

## ARGO resources

- [GitHub pyARGO](https://github.com/castelao/pyARGO) is a Python library for ARGO; status uncertain
- [UCSD ARGO Informational FAQ](http://www.argo.ucsd.edu/Data_FAQ.html#RorD)
- [Coriolis map-based data selection tool](http://www.argodatamgt.org/Access-to-data/Argo-data-selection)
- [xarray lesson that works with ARGO data](https://rabernat.github.io/research_computing/xarray.html)

## Coriolis map interface

Here is the map interface from **Coriolis** including the yellow bounding box and some green dot ARGO profiles.


<BR>
<img src="./argomap1.png" style="float: left;" alt="drawing" width="750"/>
<div style="clear: left"><BR>
    
Here is the "results" feedback when the order completes.
    
<BR>
<img src="./argomap2.png" style="float: left;" alt="drawing" width="750"/>
<div style="clear: left"><BR>

Coriolis finishes the data preparation in a matter of a minute or so. They tell you in advance the URL of 
your results, for example: 

```
We are processing your data request.
The data will be delivered: ftp://ftp.ifremer.fr/ifremer/coriolis/tmp/co0501/DataSelection_20180206_225551_5097076.tgz
```

## Order and retrieve an ARGO dataset at Coriolis 

- Use the map interface to navigate to a region of interst
- Set (as above, yellow rectangle) a bounding box 
- Use the time controls to set a time range (I used Jun 1 -- Sep 1 2019)
- Click on Refresh (in the control at the left of the map), wait for the Query to complete
- Click Download to open the Data Selection tab in your browser. 
  - This click seems a bit inconsistent... might need a second try
- Enter your email, click 'Extract your data': Produces the future URL of your dataset
- You will receive a 'processing' confirmation email
- You will receive an email with a link to the dataset when it is in place

The URL has this format:

```
ftp://ftp.ifremer.fr/ifremer/coriolis/tmp/co0501/DataSelection_yyyymmdd_hhmmss_xxxxxx.tgz
```

This file expires and vanishes in an hour or two. So grab that data before it evaporates; 
and then put it somewhere stable.

## Python ftp data pull from Coriolis

In [3]:
# This code (suitably modified) would pull data from Coriolis via ftp via in turn Python
run_ARGO_ftp = True

if run_ARGO_ftp: 
    # My bounding box is the region of the Regional Cabled Array. Time = June 1 - Sep 1 2019.
    from ftplib import FTP
    dataIDString = '20200129_190718_9538702'
    myIDString = './ARGO_Jan1_2018_Jan28_2020_Part3of3.tgz'         # This is arbitrary, to remember what I ordered
    ftp = FTP('ftp.ifremer.fr')
    ftp.login()
    ftp.cwd('ifremer/coriolis/tmp/co0501/')
    ftp.retrlines('LIST DataSelection_' + dataIDString + '.tgz')
    ftp.retrbinary('RETR DataSelection_' + dataIDString + '.tgz', open(myIDString, 'wb').write)
    ftp.quit()

error_perm: 550 Failed to open file.

## Unpack ARGO data

Suppose the data file is (for simplicity) **x.tgz**. I then run

```
$ gunzip x.tgz
$ tar -xvf x.tar
$ cd <un-tar directory>
$ ...etcetera...
```

This gets the profile and transect files -- one for each drifter in the bounding box -- into my system as
NetCDF files. In my case these are written to `~/data/argo` as `argo_profiles1.nc` and `argo_trajectory1.nc`. 


To formalize this with code here are some tools. These would be useful for a large campaign of ARGO analysis.


```
import os

def ls(qual): return os.popen('ls -al ' + qual).readlines()
    
datasetList = []
r = ls('*.tgz')
for a in r: 
    c = a.split(' ')[8].split('.')[0]
    if c not in datasetList: datasetList.append(c)
        
print(str(datasetList))

for c in datasetList:
    retvalue = os.popen('gunzip ' + c + '.tgz').readlines()
    retvalue = os.popen('tar -xvf ' + c + '.tar').readlines()
```
